In [1]:
import numpy as np 
import pygame 
from datetime import datetime
# import sys
# sys.path.append(r'C:\Users\lenovo\Desktop\Artificial Inteligence\Jupyter projects\Time Dilation\titles')
from functions import *
from light_signals import *
from image import *
from titles import titles_collection
from collections import deque

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
Global animation variables

""" 

screen_width = 1500
screen_height = 800
FPS = 100     # Frames per second

# Initial distance between moving body and stationary observer
distance = 1300
# Velocity is measured as a percentage of the speed of light - [0, 1) 
velocity = 0.85

rocket_image_path = r'C:\Users\lenovo\Desktop\Artificial Inteligence\Jupyter projects\Time Dilation\images\rocket.jfif'
receiver_image_path = r'C:\Users\lenovo\Desktop\Artificial Inteligence\Jupyter projects\Time Dilation\images\receiver.png'

In [3]:
# titles_collection = {
#     0: 'Let a spaceship move through space at 85 percent of the speed of light. At a certain\nposition ahead of it, there is a receiver whose function is to receive signals,\nrecord the exact time of reception of these signals, and the messages they carry.\nLet the spaceship start emitting signals at regular intervals of 1 second according\nto its own clock when it is exactly 15 light-seconds away from the receiver.\nEach signal carries a message that contains information about the moment it was emitted.\nThus, message 0 indicates that the signal was emitted at time 0, message 1 at time 1, and so on.'
# }

In [9]:
def main_animation(screen_width, screen_height,FPS):
    
    '''
    Initializations
    '''
    pygame.init()
    bg_color = (255, 255, 255)
    line_color = (185, 185, 185) # Color of the coordinate line
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer
    data_layer = pygame.Surface((screen_width, screen_height), pygame.SRCALPHA) # Data layer
    title_layer = pygame.Surface((screen_width, screen_height), pygame.SRCALPHA)
    clock = pygame.time.Clock()
    pygame.font.init()
    font_big = pygame.font.Font(None, 36)
    font_small = pygame.font.Font(None, 26)
    message_font = pygame.font.Font(None, 20)
    titles_font = pygame.font.SysFont('arial', 36)
    
    units =15 # Every unit is equal to 1 light second
    
    length = distance/units # Length of one unit in pixels
    
    # The animation layer responsible for the signals data
    background_surface, markers, screen_objects = create_background(screen_width, screen_height, bg_color, line_color, font_small, units, length)  
    LightSignalClass = LightSignal
    LightSignal.receiver_position = {k: v for k, v in markers[0].items()}
    
    # Create rocket object with image
    rocket = Image(rocket_image_path, [0, 500])
    rocket.image = pygame.transform.rotate(rocket.image, -90)
    rocket.image = pygame.transform.scale(rocket.image, (90, 90))
    
    # Create receiver object with image
    receiver = Image(receiver_image_path, [screen_width - 90, 600])
    receiver.image = pygame.transform.scale(receiver.image, (70, 70))
    
    all_sprites = pygame.sprite.Group()
    all_sprites.add(rocket)
    all_sprites.add(receiver)
    
    background_surface.blit(receiver.image, receiver.rect)
    win.blit(rocket.image, rocket.rect)
    
    
    '''
    Local variables
    '''
    
    is_receiver_chron_started = False
    receiver_frames_count = 0 # Tracks the frames when the receiver starts his chronometer   
    
    displacement_per_frame = length/ FPS # The distance that light travels for 1 second
    
    frame = -1
    start_angle = - np.pi/16 
    end_angle = - start_angle # Boundaries of the signals fronts
    
    signal_radius = 0
    
    gamma_fac = gamma_factor(velocity) # Calculates how much slower the clock on the spaceship runs
    
    # The front of the first light signal emmited from the ship.
    # It will be used for the calculations of the spaceship position    
    first_light_front = [100, 500]
    space_ship_pos = first_light_front.copy()   
      
    emitted_signals = deque([]) # Contains the emitted from spaceship singles objects
    
    emitted_signals.append(LightSignalClass.create_signal(space_ship_pos.copy(), '0'))
    
    readings_dict = {} # Readings of the receiver
       
    spaceship_tickings = 1
    messages_count = 0

    running = True
    
    is_animation_stopped = False
    
    after_stop = False
    
    while running:
        
        clock.tick(FPS)        
      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
        if not is_animation_stopped:            
           
            frame += 1
            tick = pygame.time.get_ticks()
            data_layer.fill((0, 0, 0, 0))
            text = font_big.render(f'Frame number {frame} - time {tick}', True, (0,0,0))

            space_ship_time = (frame * gamma_fac) / FPS

            stationary_time = frame / FPS

            space_ship_text = font_big.render(f'Spaceship clock {space_ship_time:.2f}', True, (0, 0, 0))
            stationary_text = font_big.render(f'Stationary clock {stationary_time:.2f}', True, (0, 0, 0))

            received_time_text = font_big.render('Received time', True, (0, 0, 0))
            received_message_text = font_big.render('Received message', True, (0, 0, 0))
            
            # Print needed data on the screen
            win.blit(background_surface, (0, 0))     
            data_layer.blit(text, (10, 10))
            data_layer.blit(space_ship_text, (10, 80))
            data_layer.blit(stationary_text, (10, 50))
            data_layer.blit(received_time_text, (1000, 10))
            data_layer.blit(received_message_text, (1250, 10))   

            first_light_front = update_obj_position(first_light_front, displacement_per_frame)

            # pygame.draw.circle(win, (0, 0, 0),first_light_front , radius=2)
            space_ship_pos = update_obj_position(space_ship_pos, displacement_per_frame * velocity)
                        
            screen_objects['spaceship_last_position'] = space_ship_pos
            
            rocket.position = [space_ship_pos[0] - 90, space_ship_pos[1] - 0]
            
            win.blit(rocket.image, rocket.position)

            # Remove the the first signal in the queue when is reaching the receiver and
            # get data from it.
            if emitted_signals and emitted_signals[0].is_receiver_reached:
                if not is_receiver_chron_started:
                    is_receiver_chron_started = True
                chronometer_time = receiver_frames_count / FPS
                signal = emitted_signals.popleft()
                readings_dict = update_readings(readings_dict, len(readings_dict),chronometer_time, signal.message)

            if readings_dict:
                show_readings(readings_dict, font_big, data_layer)                

            # Active when the receiver`s clock is started
            if is_receiver_chron_started:

                chronometer_time = receiver_frames_count / FPS
                chronometer_time_text = font_big.render(f'Stationary chronometer {chronometer_time:.2f}', True, (0, 0, 0))
                receiver_frames_count += 1

                data_layer.blit(chronometer_time_text, (10, 120))


            # Update the position of every emitted signal
            for signal in emitted_signals:
                signal.update_signal_position(displacement_per_frame)

                message_coords = update_message_position(end_angle, signal, after_stop)          

                message = message_font.render(signal.message, True, (128,128, 128))
                data_layer.blit(message, message_coords)

                pygame.draw.arc(win, (255, 255, 0), signal.rect, start_angle, end_angle, 2)
                pygame.draw.circle(win, (227, 18, 233), signal.origin_position, radius = 2)


            # Draw spaceship position
            pygame.draw.circle(win, (255, 0, 0), space_ship_pos, radius = 2)
            
            win.blit(data_layer, (0, 0))
        
            if space_ship_time >= spaceship_tickings:
                signal_point = pygame.draw.circle(background_surface, (255, 255, 0), space_ship_pos, radius = 2)

                # Add signal start position to screen_objects['signals_start_positions']
                if spaceship_tickings not in screen_objects['signals_start_positions'].keys():
                    screen_objects['signals_start_positions'][spaceship_tickings] = space_ship_pos.copy()
             
                messages_count += 1
                emitted_signals.append(LightSignalClass.create_signal(space_ship_pos.copy(), str(messages_count)))

                spaceship_tickings += 1
        
        # If the spaceship is reached the receiver
        if space_ship_pos[0] >= screen_objects['x_line_end']:
            
            is_animation_stopped = True            

            first_light_front[0] -= 20 / velocity
            screen_objects = shift_screen_objects(screen_objects, -20)
           
            for i in range(units+1):
                
                # Clear the background surface and win before drawing new lines
                win.fill(bg_color)
                background_surface.fill(bg_color)
                
                # Draw horizontal coordinate line
                pygame.draw.line(background_surface, color = line_color, start_pos=(screen_objects['x_line_start'], screen_objects['y_line']),
                         end_pos = (screen_objects['x_line_end'], screen_objects['y_line']), width=2)
                                    
                # Draw the shifting coordinate line
                for i in range(len(screen_objects['numbers'])):
                    # Draw numbers below the coordinate line
                    number, x_coord, y_coord = screen_objects['numbers'][i]
                    number_text = font_small.render(f"{number}", True, line_color)
                    background_surface.blit(number_text, (x_coord, y_coord))
                    
                    
                    # Draw vertical lines on the horizontal line
                    pygame.draw.line(background_surface, color = line_color, 
                             start_pos=(screen_objects['vertical_line_x'][i], screen_objects['vertical_line_start']),
                             end_pos = (screen_objects['vertical_line_x'][i], screen_objects['vertical_line_end']), width=1)
                    
                
                description = font_small.render(f'Distance in light seconds', True, line_color)
                background_surface.blit(description, (650, 700))
                
                
                pygame.draw.circle(background_surface, (255, 0, 0), screen_objects['spaceship_last_position'], radius = 2)
                
                rocket.position = [screen_objects['spaceship_last_position'][0] - 90, screen_objects['spaceship_last_position'][1] - 0]
                receiver.position = [screen_objects['spaceship_last_position'][0] +10, 600]
                
                background_surface.blit(receiver.image, receiver.position)
                
                win.blit(background_surface, (0,0))
                win.blit(data_layer, (0, 0))
                win.blit(rocket.image, rocket.position)
               
             
            for _, positions in screen_objects['signals_start_positions'].items():                

                pygame.draw.circle(win, (255, 0, 0), positions, radius = 2)
               
            if screen_objects['x_line_end'] <= screen_width-1400:
                is_animation_stopped = False
                
                # Move the position of the receiver at the starting point of the coordinate line
                LightSignal.receiver_position['receiver_x'] = screen_width-1400
                
                # Update spaceship_pos
                space_ship_pos = screen_objects['spaceship_last_position'].copy()
                
                # Change the bondaries of the signal`s arc
                start_angle = 15 * np.pi/16
                end_angle = 17 * np.pi/16
                
                after_stop = True
                
                receiver.position = [screen_objects['spaceship_last_position'][0] - 70, 600]
                receiver.image = pygame.transform.flip(receiver.image, True, False)
                
                # New background with coordinate numbers from 0 to 15
                background_surface, markers, screen_objects = create_background(screen_width, screen_height, bg_color, line_color, font_small, units, length, after_stop = True)
                background_surface.blit(receiver.image, receiver.position)
                
                
                screen_objects['x_line_end'] = screen_width - 100
        
        if frame in titles_collection:
            print(titles_collection)
            show_titles(win, title_layer,titles_font, titles_collection[frame])
        
        pygame.display.update()
        pygame.display.flip()
    # Exit
    pygame.quit()

        

In [10]:
main_animation(screen_width, screen_height,FPS)

{0: {'a': ['Let a spaceship move through space at 85 percent of the speed of light. At a certain\nposition ahead of it, there is a receiver whose function is to receive signals,\nrecord the exact time of reception of these signals, and the messages they carry.\nLet the spaceship start emitting signals at regular intervals of 1 second according\nto its own clock when it is exactly 15 light-seconds away from the receiver.\nEach signal carries a message that contains information about the moment it was emitted.\nThus, message 0 indicates that the signal was emitted at time 0, message 1 at time 1, and so on.', 30]}}
{'a': ['Let a spaceship move through space at 85 percent of the speed of light. At a certain\nposition ahead of it, there is a receiver whose function is to receive signals,\nrecord the exact time of reception of these signals, and the messages they carry.\nLet the spaceship start emitting signals at regular intervals of 1 second according\nto its own clock when it is exactly 1

In [6]:
len('Let a spaceship move through space at 85 percent of the speed of light. At a certain')

84

In [7]:
expr = 'Each signal carries a message that contains information about the moment it was emitted. Thus, message 0 indicates that the signal was emitted at time 0, message 1 at time 1, and so on.'

In [8]:
for i in range(len(expr)):
    if i % 84 == 0:
        print()
    print(expr[i], end='')


Each signal carries a message that contains information about the moment it was emit
ted. Thus, message 0 indicates that the signal was emitted at time 0, message 1 at t
ime 1, and so on.